# Handwritten Digit Recognition using Python

In [1]:
import keras #Importing Keras
from keras.datasets import mnist #Importinh MNIST dataset from the inbuilt datasets in Keras
from keras.models import Sequential #Importing Sequencial Model
from keras.layers import Dense, Dropout, Flatten    #Importing dense, dropout and flatten layers
from keras.layers import Conv2D, MaxPooling2D   #Importing Conv2D and MaxPooling2D
from keras import backend as K  #Importing backend from keras


# Splitting the data between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(x_train.shape, y_train.shape) #Printing the dimensions of training data

(60000, 28, 28) (60000,)


In [2]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
#Reshaping thw z_train and x_test data based on the needs
input_shape = (28, 28, 1)
#Storing the input shape of the training data in a variable
num_classes = 10

# convert class vectors to binary class matrices
from keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
#Converting y_train and y_test to num_classes


x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
#Converting x_train & x_test into float of 32 bits

x_train /= 255
x_test /= 255
#Dividing x_train & x_test with 255 as it is the maximum value of byte so this will ensure that the input features are scaled between 0.0 and 1.0.


#printing x_train shape and number of samples in x_train & x_test data
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [3]:
batch_size = 128
num_classes = 10
epochs = 10
#Defining batch_size, bun_classes and epochs

#Creating the model
model = Sequential()
#Adding a Conv2D layer with 32 filters, defining kernel size,activation function and input shape
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape)) 
model.add(Conv2D(64, (3, 3), activation='relu'))    #Adding a Conv2D layer with 64 filters
model.add(MaxPooling2D(pool_size=(2, 2)))           #Adding a MaxPooling2D layer
model.add(Dropout(0.25))                            #Adding a Dropout layer
model.add(Flatten())                                #Adding the Flatten layer
model.add(Dense(256, activation='relu'))            #Adding the dense layer
model.add(Dropout(0.5))                             #Adding another Dropout layer
model.add(Dense(num_classes, activation='softmax')) #Adding Dense layer

model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])    #Compiling the model using Adadeltaoptimizer

In [4]:
hist = model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))   #Training the model
print("The model has successfully trained")

model.save('mnist.h5')
print("Saving the model as mnist.h5")   #Saving the model to mnist.h5

Epoch 1/10
469/469 [==============================] - 65s 138ms/step - loss: 2.2949 - accuracy: 0.1284 - val_loss: 2.2811 - val_accuracy: 0.2328
Epoch 2/10
469/469 [==============================] - 75s 160ms/step - loss: 2.2714 - accuracy: 0.2139 - val_loss: 2.2534 - val_accuracy: 0.3808
Epoch 3/10
469/469 [==============================] - 79s 169ms/step - loss: 2.2427 - accuracy: 0.3020 - val_loss: 2.2176 - val_accuracy: 0.5081
Epoch 4/10
469/469 [==============================] - 78s 167ms/step - loss: 2.2032 - accuracy: 0.3784 - val_loss: 2.1670 - val_accuracy: 0.5927
Epoch 5/10
469/469 [==============================] - 79s 169ms/step - loss: 2.1459 - accuracy: 0.4453 - val_loss: 2.0921 - val_accuracy: 0.6493
Epoch 6/10
469/469 [==============================] - 80s 171ms/step - loss: 2.0609 - accuracy: 0.5015 - val_loss: 1.9803 - val_accuracy: 0.6947
Epoch 7/10
469/469 [==============================] - 81s 172ms/step - loss: 1.9381 - accuracy: 0.5508 - val_loss: 1.8171 - val_ac

In [5]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])                       #Evaluating the score of the model and printing the test loss 7 test accuracy
print('Test accuracy:', score[1])

Test loss: 1.1044260263442993
Test accuracy: 0.8057000041007996


In [6]:
#Digit Classifier GUI


#Importing libraries
from keras.models import load_model
from tkinter import *
import tkinter as tk
from win32 import win32gui
from PIL import ImageGrab, Image
import numpy as np

model = load_model('mnist.h5') #Loading the previously trained model

def predict_digit(img):
    #resize image to 28x28 pixels
    img = img.resize((28,28))
    #convert rgb to grayscale
    img = img.convert('L')
    img = np.array(img)
    #reshaping to support our model input and normalizing
    img = img.reshape(1,28,28,1)
    img = img/255.0
    #predicting the class
    res = model.predict([img])[0]
    return np.argmax(res), max(res)

class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0

        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg = "white", cursor="cross")
        self.label = tk.Label(self, text="Thinking..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text = "Recognise", command =         self.classify_handwriting) 
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)

        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label.grid(row=0, column=1,pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)

        #self.canvas.bind("<Motion>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def clear_all(self):
        self.canvas.delete("all")

    def classify_handwriting(self):
        HWND = self.canvas.winfo_id() # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND) # get the coordinate of the canvas
        im = ImageGrab.grab(rect)

        digit, acc = predict_digit(im)
        self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')

    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')

app = App()
mainloop()